## Webscraping notebook

In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [80]:
#loading downloaded data
df=pd.read_json('~/Downloads/reviews_Electronics_5.json', lines=True)

In [81]:
#examining text of reviews
df.reviewText

0          We got this GPS for my husband who is an (OTR)...
1          I'm a professional OTR truck driver, and I bou...
2          Well, what can I say.  I've had this unit in m...
3          Not going to write a long review, even thought...
4          I've had mine for a year and here's what we go...
                                 ...                        
1689183    Burned these in before listening to them for a...
1689184    Some people like DJ style headphones or earbud...
1689185    I&#8217;m a big fan of the Brainwavz S1 (actua...
1689186    I've used theBrainwavz S1 In Ear Headphones, a...
1689187    Normally when I receive a review sample I can ...
Name: reviewText, Length: 1689188, dtype: object

In [4]:
#error asins
# df= df[df.asin!='B00004SYNX']
# df= df[df.asin!='B00000J1SC']
# df= df[df.asin!='B000067RVL']
# df= df[df.asin!='B000068O53']
# df= df[df.asin!='B00006B8EC']
# df= df[df.asin!='']

### The links, links1, and links2 pkl files contain scraped links of the Amazon products. They are error free.

In [ ]:
url=

In [57]:
with open ('links.pickle', 'wb') as f:
    pickle.dump(links, f, 5)

In [67]:
with open ('links1.pickle', 'wb') as f:
    pickle.dump(links1, f, 5)

In [75]:
with open ('links2.pickle', 'wb') as f:
    pickle.dump(links2, f, 5)

### Webscraping the item name, item tags, and price to be added to df

In [125]:
#getting our urls ready for webscraping
df1= pd.Series(links)
df2=pd.Series(links1)
df3=pd.Series(links2)
frames=[df1,df2,df3]
url_series=pd.concat(frames, axis=0)
len(url_series)

26885

### Scraping the item name, item tag, and price of the item

In [ ]:
from tqdm import tqdm
import time
itemz=[]
linkz_p=[]
tagz_p=[]
item_namezz=[]
driver = webdriver.Chrome()
for x in tqdm(links):
    try:
        driver.get(str(x))
        content=driver.find_elements_by_class_name('a-price')
        content1=driver.find_elements_by_class_name('a-link-normal.a-color-tertiary')
        content2=driver.find_elements_by_id('productTitle')
        see= [x.text for x in content]
        item= [x.text for x in content1]
        item_namez= [x.text for x in content2]
        itemz.append(item)
        tagz_p.append(see)
        item_namezz.append(item_namez)
        linkz_p.append(x)
    except:
        continue
        time.sleep(.0005)
        driver.close()

  0%|                                                 | 0/26885 [00:00<?, ?it/s]/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_1871/3200647733.py:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content=driver.find_elements_by_class_name('a-price')
/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_1871/3200647733.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content1=driver.find_elements_by_class_name('a-link-normal.a-color-tertiary')
/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_1871/3200647733.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content2=driver.find_elements_by_id('productTitle')
 62%|█████████████████████▋             | 16659/26885 [01:29<00:10, 1019.52it/s]

In [195]:
#cleaning prices
prices=[]
for x in tagz_p:
    if len(x) > 1:
        prices.append(x[0])
    else:
        prices.append(x)


In [197]:
cleaned_prices=[str(x).replace('\n', '.') for x in prices]

In [203]:
cleaned_prices=[str(x).replace('[', '') for x in cleaned_prices]

In [210]:
cleaned_prices=[str(x).replace(']', '') for x in cleaned_prices]

In [212]:
cleaned_prices=[str(x).replace("'", '') for x in cleaned_prices]

In [273]:
cleaned_prices=pd.Series(cleaned_prices)

In [241]:
#cleaning tags
items=pd.Series(itemz)
test=[items.iloc[x] for x in range(len(items))]


In [257]:
tags=[test[i][-1] for i in range(len(test)) if len(test[i])>=1]
tags=[]
for i in range(len(test)):
    if len(test[i]) >=1:
        tags.append(test[i][-1])
    else:
        tags.append(np.nan)

In [274]:
tags=pd.Series(tags)

In [270]:
#cleaning links to join on asin
links=[str(x)[-10:] for x in linkz_p]

In [275]:
links=pd.Series(links)

In [282]:

df_scrape=pd.concat([links, tags, cleaned_prices], axis=1)

In [284]:
df_scrape.rename(columns={0:'asin', 1:'tag', 2:'price'})

,asin,tag,price
0,0528881469,Trucking GPS,$399.99
1,0594451647,Chargers & Adapters,$29.99
2,0594481813,Power Adapters,$16.99
3,0972683275,TV Ceiling & Wall Mounts,$34.99
4,1400501466,Tablets,$49.99
...,...,...,...
26862,B003JUN9YW,Over-Ear Headphones,$65.65
26863,B003JUXWSU,Covers,
26864,B003JV3XZ6,HDMI Cables,$9.74
26865,B003JVTOA4,Camera Cases,$22.54


In [294]:
load_df=df.merge(df_scrape, how='inner', left_on='asin', right_on='asin')

KeyError: 'asin'

In [295]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011"
...,...,...,...,...,...,...,...,...,...
1689183,A34BZM6S9L7QI4,B00LGQ6HL8,"Candy Cane ""Is it just me?""","[1, 1]",Burned these in before listening to them for a...,5,Boom -- Pop -- Pow. These deliver.,1405555200,"07 17, 2014"
1689184,A1G650TTTHEAL5,B00LGQ6HL8,"Charles Spanky ""Zumina Reviews""","[0, 0]",Some people like DJ style headphones or earbud...,5,"Thin and light, without compromising on sound ...",1405382400,"07 15, 2014"
1689185,A25C2M3QF9G7OQ,B00LGQ6HL8,Comdet,"[0, 0]",I&#8217;m a big fan of the Brainwavz S1 (actua...,5,Same form factor and durability as the S1 with...,1405555200,"07 17, 2014"
1689186,A1E1LEVQ9VQNK,B00LGQ6HL8,J. Chambers,"[0, 0]","I've used theBrainwavz S1 In Ear Headphones, a...",5,Superb audio quality in a very comfortable set...,1405641600,"07 18, 2014"
